In [1]:
from ultralytics import YOLO
import numpy as np
import os
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\minsub\.conda\envs\bax\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### tree

In [2]:
# 나무의 바운딩 박스 크기로 판별하는 함수
def tree_size(results, checking_class): # 욜로 모델로 예측된 결과값과 각 객체들의 예측 클래스 번호가 담긴 리스트를 인자로 받음
    # 검출된 객체들을 순회
    for result in results: 
        boxes = result.boxes # 각 객체의 바운딩박스 정보를 boxes로 정의

    trees =[] # 객체들이 담긴 리스트에서 나무로 판별된 객체 리스트의 인덱스 번호를 담을 리스트
    bbox_sizes = [] # 바운딩 박스의 크기 정보를 담을 리스트

    for idx, data in enumerate(checking_class): # 탐지된 객체들의 클래스 정보가 담긴 리스트 
        if data == 6: # 클래스 번호가 6일시 ("나무"클래스의 번호가 6임)
            tree_num = idx # 해당 리스트의 인덱스 번호를 trees리스트에 추가 
            trees.append(tree_num)
            
    print(trees) 

    # 나무로 검출된 객체들의 바운딩 박스 넓이를 구해서 리스트에 추가하는 작업
    for tree_num in trees: # 나무로 탐지된 객체의 인덱스 번호가 담긴 리스트를 순회
        if tree_num != -1: 
            tree_xyxy_cordi = boxes.xyxy[tree_num].tolist() # 해당 객체의 바운딩 박스의 좌상단 우하단 좌표
            bbox_size = (tree_xyxy_cordi[2] - tree_xyxy_cordi[0]) * (tree_xyxy_cordi[3] - tree_xyxy_cordi[1]) # 좌표를 통해 바운딩 박스의 가로 세로 길이를 구해 넓이를 구함
            bbox_sizes.append(bbox_size) # 구한 바운딩 박스 넓이를 리스트에 추가

    # 가장 큰 바운딩 박스의 인덱스 찾기
    largest_bbox_idx = bbox_sizes.index(max(bbox_sizes))

    print("가장 큰 바운딩 박스의 인덱스:", largest_bbox_idx)

    tree_num = trees[largest_bbox_idx] # 가장 넓이가 큰 바운딩 박스의 인덱스 번호로 정보 반환
    tree_num

    branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size = tree_assess(tree_num, results, checking_class) # 가장큰 나무의 정보를 받아 각 요소들 라벨링하는 함수

    return branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size

In [3]:
# 검출된 객체간의 iou를 계산하는 함수
def cal_iou(boxA, boxB):
    
    # boxA와 boxB의 x, y 좌표 값 중 큰 값을 선택하여 IoU 계산에 사용할 시작 좌표를 결정
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    
    # boxA와 boxB의 x, y 좌표 값 중 작은 값을 선택하여 IoU 계산에 사용할 끝 좌표를 결정
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # 두 상자의 겹치는 영역의 넓이 계산
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # 각 상자의 전체 영역의 넓이 계산
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # iou계산
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

In [4]:
# 가장 큰 bbox 나무의 인덱스 번호, 객체들 정보, 객체들의 클래스 리스트를 받아 각 요소들을 라벨링하는 함수
def tree_assess(tree_num, results, checking_class):
    for result in results: # 탐지된 객체들을 순회하면서
        boxes = result.boxes # 각 객체의 bbox정보를 boxes에 할당

    tree_xyxy_cordi = boxes.xyxy[tree_num].tolist() # 바운딩 박스의 좌상단 우하단 좌표 리스트
    tree_center_x_cordi = boxes.xywh[tree_num].tolist()[0] # 바운딩 박스의 중심좌표와 가로 세로 길이 리스트

    # 가지, 뿌리, 수관, 열매, 옹이나 상처, 위치, 크기
    
    # 가지 ============================================================
    branch_class_num = [] # 가지 클래스의 인덱스 번호를 담을 리스트
    for idx, data in enumerate(checking_class): # 객체들의 클래스 번호가 담긴 리스트를 순회하면서
        if data == 0 or data == 1: # 리스트에 담긴 클래스 번호가 0 or 1 일시 (원래는 0인데 라벨링 실수로 branch_yn, branch_ynw 2개가 학습됨)
            branch_class_num.append(idx) # 해당 인덱스 번호를 branch_class_num에 추가

    branch_xyxy_cordi = [] # 가지 바운딩 박스의 좌표 정보를 담을 리스트
    for idx in branch_class_num: # 가지의 인덱스 번호가 담긴 리스트를 순회 하면서
        branch_xyxy_cordi.append(boxes.xyxy[idx].tolist()) # 가지의 바운딩 박스 좌상단 우하단 좌표 리스트를 리스트에 추가

    branch_iou = [] # 가지의 iou 결과값을 담을 리스트
    for i in branch_xyxy_cordi: # 가지의 바운딩 박스 정보가 담긴 리스트를 순회
        branch_iou.append(cal_iou(tree_xyxy_cordi,i)) # 나무 바운딩 박스와 가지 바운딩 박스의 iou를 계산한 값을 리스트에 추가

    branch_count = 0 # 가지의 개수 초기화
    for i in branch_iou: # 가지와 나무의 iou값을 순회
        if i != 0: # iou값이 0이 아니면(iou가 0일시 나무와 가지가 겹치지않음 -> 나무로 판단된 영역안에 가지가 없기에 가지로 판단된 객체가 잘못 된것이라 예측해 카운팅X)
            branch_count += 1 # 가지의 개수 +1

    branch_yn = '' # 가지 라벨링 정보 초기화
    if branch_count == 0: # 가지의 카운팅수가 0일시 
        branch_yn = 'n' # 가지의 라벨링을 'n'으로 없다라고 판정
    else: # 
        branch_yn = 'y' # 가지의 라벨링을 'y'으로 있다라고 판정

    # 위와 같은 과정으로 나머지 요소들도 iou를 바탕으로 나무로 판정된 영역 내에 겹치지 않는 것을 제외하고 카운팅해 라벨링
    
    # 뿌리 ============================================================
    root_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 5:
            root_class_num.append(idx)

    root_xyxy_cordi = []
    for idx in root_class_num:
        root_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    root_iou = []
    for i in root_xyxy_cordi:
        root_iou.append(cal_iou(tree_xyxy_cordi,i))

    root_count = 0
    for i in root_iou:
        if i != 0:
            root_count += 1

    root_yn = ''
    if root_count == 0:
        root_yn = 'n'
    else:
        root_yn = 'y'

    # 수관 ============================================================
    crown_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 2:
            crown_class_num.append(idx)

    crown_xyxy_cordi = []
    for idx in crown_class_num:
        crown_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    crown_iou = []
    for i in crown_xyxy_cordi:
        crown_iou.append(cal_iou(tree_xyxy_cordi,i))

    crown_count = 0
    for i in crown_iou:
        if i != 0:
            crown_count += 1

    crown_yn = ''
    if crown_count == 0:
        crown_yn = 'n'
    else:
        crown_yn = 'y'

    # 열매 ============================================================
    fruit_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 3:
            fruit_class_num.append(idx)

    fruit_xyxy_cordi = []
    for idx in fruit_class_num:
        fruit_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    fruit_iou = []
    for i in fruit_xyxy_cordi:
        fruit_iou.append(cal_iou(tree_xyxy_cordi,i))

    fruit_count = 0
    for i in fruit_iou:
        if i != 0:
            fruit_count += 1

    fruit_yn = ''
    if fruit_count == 0:
        fruit_yn = 'n'
    else:
        fruit_yn = 'y'

    # 옹이나 상처 ============================================================
    gnarl_class_num = []
    for idx, data in enumerate(checking_class):
        if data == 4:
            gnarl_class_num.append(idx)

    gnarl_xyxy_cordi = []
    for idx in gnarl_class_num:
        gnarl_xyxy_cordi.append(boxes.xyxy[idx].tolist())

    gnarl_iou = []
    for i in gnarl_xyxy_cordi:
        gnarl_iou.append(cal_iou(tree_xyxy_cordi,i))

    gnarl_count = 0
    for i in gnarl_iou:
        if i != 0:
            gnarl_count += 1

    gnarl_yn = ''
    if gnarl_count == 0:
        gnarl_yn = 'n'
    else:
        gnarl_yn = 'y'

    # 위치와 크기 ============================================================
    image_w = boxes.orig_shape[0] * 0.965 # 입력받은 이미지 width
    image_h = boxes.orig_shape[1] # 입력받은 이미지 height
    image_size = image_w * image_h # 입력받은 이미지 넓이
    tree_size = (tree_xyxy_cordi[2] - tree_xyxy_cordi[0]) * (tree_xyxy_cordi[3] - tree_xyxy_cordi[1]) # 나무 bbox의 넓이
    tree_size_percent = tree_size / image_size * 100 # 입력받은 이미지 대비 나무의 bbox가 몇% 차지하는지 계산
    
    # 크기
    size = '' # 크기 라벨링 정보 초기화
    if tree_size_percent < 18 * 0.8: # 면적비율이 (18 * 0.8)% 미만일시
        size = 'small' # small
    elif tree_size_percent >= 18 * 0.8 and tree_size_percent < 55 * 0.8: # 면적비율이 (18 * 0.8)% ~ (55 * 0.8)% 미만일시
        size = 'middle'
    elif tree_size_percent >= 55 * 0.8: # 면적비율이 (55 * 0.8) 초과일시
        size = 'big'
    
    # 위치
    loc = '' # 위치 라벨링 정보 초기화
    if tree_center_x_cordi < image_w/3: # 입력받은 이미지의 가로길이를 3등분해서 1/3지점보다 앞에 나무의 bbox의 중심좌표가 있을시
        loc = 'left'
    elif tree_center_x_cordi >= image_w/3 and tree_center_x_cordi < image_w/3*2: # 1/3 ~ 2/3지점사이에 있으시 
        loc = 'center'
    elif tree_center_x_cordi >= image_w/3*2: # 2/3지점 보다 뒤에 있을시
        loc = 'right'
    
    return branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size

In [5]:
# 나무로 탐지된 객체수에 따라 라벨링을 결정
def tree_csv(results): # 객체들의 정보를 인자로 받음
    for result in results: # 객체들을 순회하면서
        boxes = result.boxes # 각 객체의 bbox 정보를 boxes로 할당
        
    checking_class = boxes.cls.int().tolist() # 각 bbox의 클래스 번호를 리스트로 반환

    tree_num = -1 # 나무 인덱스 번호 초기화
    
    for idx, data in enumerate(checking_class): # 클래스 번호가 담긴 리스트를 순회하며
        if data == 6: # 클래스 번호가 6일시 (나무 클래스 번호가 6)
            tree_num = idx # 해당 인덱스 번호를 할당후 for문 탈출
            break
    
    if tree_num != -1: # 나무 인덱스 번호가 -1이 아닐시 (나무로 검출된 객체가 있음)
        if checking_class.count(6) == 0: # 객체들의 클래스 번호가 담긴 리스트에서 6이 0개 일시
            print('나무가 검출되지 않았습니다.') 
        else:
            # 크기가 가장 큰 바운딩 박스 찾는 함수 호출
            branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size = tree_size(results, checking_class)
    else: # 오류 방지 차원 알고리즘으로 라벨링 불가시
        branch_yn = 'no detect'
        root_yn = 'no detect'
        crown_yn = 'no detect'
        fruit_yn = 'no detect'
        gnarl_yn = 'no detect'
        loc = 'no detect'
        size = 'no detect'

    return branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size

In [6]:
model = YOLO('./tree_x_weight/weights/best.pt') # 모델 로드

In [7]:
def yolov8_tree_detect(path,filename): # 검사할 이미지의 경로와 파일 이름을 인자로
    
    results = model.predict(path+ '/' + filename) # 로드한 모델로부터 예측결과를 results에 할당
    
    res_plot = results[0].plot() 
    
    plt.imshow(res_plot)

    branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size = tree_csv(results)
    
    return branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size

In [8]:
def tree_save(img_dir, save_dir):
    
    all_images = [f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f)) and f.endswith(('.png', '.jpg', '.jpeg'))] 

    filenames = []

    tree_df = pd.DataFrame(columns=('id', 'branch_yn', 'root_yn', 'crown_yn', 'fruit_yn', 'gnarl_yn', 'loc', 'size')) # 컬럼명을 지정한 빈 데이터프레임 생성

    for i in all_images:

        filenames.append(i.split('.')[0][:-4])

    for filename, i in tqdm(zip(all_images, filenames)): 
        branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size = yolov8_tree_detect(img_dir, filename)
        output_filename =save_dir + i + '.jpg'
        plt.savefig(output_filename, format='jpg', dpi=300)
        plt.close()
        tree_df.loc[len(tree_df)] = [i, branch_yn, root_yn, crown_yn, fruit_yn, gnarl_yn, loc, size] # 행기준으로 추가
    
    tree_df.to_csv(save_dir +'tree_result.csv', index=False) # csv파일로 저장

In [9]:
tree_save(img_dir='./tree.v1i.yolov8/test/images', save_dir= './tree_result/')

0it [00:00, ?it/s]
image 1/1 c:\yolov8\tree.v1i.yolov8\test\images\24_175_23183_tree_jpg.rf.d122b20c6241e2e178f6f68256b36a3c.jpg: 640x640 1 crown_yn, 1 tree, 51.0ms
Speed: 4.0ms preprocess, 51.0ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
1it [00:03,  3.27s/it]


[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\tree.v1i.yolov8\test\images\24_176_23041_tree_jpg.rf.825b781ae7699229b9cebda9fb71b26b.jpg: 640x640 5 branch_yns, 1 crown_yn, 1 fruit_yn, 1 tree, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
2it [00:03,  1.48s/it]


[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\tree.v1i.yolov8\test\images\24_176_23042_tree_jpg.rf.e025df75a3edba336a89dc991bd73332.jpg: 640x640 2 branch_yns, 1 crown_yn, 5 fruit_yns, 5 root_yns, 1 tree, 52.0ms
Speed: 1.0ms preprocess, 52.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
3it [00:03,  1.09it/s]


...

[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\tree.v1i.yolov8\test\images\5_29_23026_tree_jpg.rf.9e49df84fcb6efef37e5b16e1074ce30.jpg: 640x640 10 branch_yns, 1 crown_yn, 1 tree, 55.0ms
Speed: 2.0ms preprocess, 55.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
99it [00:27,  3.97it/s]


[0]
가장 큰 바운딩 박스의 인덱스: 0


image 1/1 c:\yolov8\tree.v1i.yolov8\test\images\5_29_23030_tree_jpg.rf.8f4394181854361d8fc1a48ff66e0aa9.jpg: 640x640 1 crown_yn, 1 root_yn, 1 tree, 49.0ms
Speed: 2.0ms preprocess, 49.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
100it [00:28,  3.56it/s]

[0]
가장 큰 바운딩 박스의 인덱스: 0


In [11]:
tree_ori_df = pd.read_csv('./dbi_tree.csv')
tree_pred_df = pd.read_csv('./tree_result/tree_result.csv')
tree_ori_100_df = tree_ori_df[tree_ori_df['id'].isin(tree_pred_df['id'])]

branch_count = 0
root_count = 0
crown_count = 0
fruit_count = 0
size_count = 0
gnarl_count = 0
loc_count = 0
for i in range(len(tree_pred_df)):
    if tree_ori_100_df['branch_yn'].tolist()[i] != tree_pred_df['branch_yn'].tolist()[i]:
        branch_count += 1
    if tree_ori_100_df['root_yn'].tolist()[i] != tree_pred_df['root_yn'].tolist()[i]:
        root_count += 1
    if tree_ori_100_df['crown_yn'].tolist()[i] != tree_pred_df['crown_yn'].tolist()[i]:
        crown_count += 1
    if tree_ori_100_df['fruit_yn'].tolist()[i] != tree_pred_df['fruit_yn'].tolist()[i]:
        fruit_count += 1
    if tree_ori_100_df['gnarl_yn'].tolist()[i] != tree_pred_df['gnarl_yn'].tolist()[i]:
        gnarl_count += 1
    if tree_ori_100_df['loc'].tolist()[i] != tree_pred_df['loc'].tolist()[i]:
        loc_count += 1
    if tree_ori_100_df['size'].tolist()[i] != tree_pred_df['size'].tolist()[i]:
        size_count += 1

print(f'branch diff : {branch_count}')
print(f'root diff : {root_count}')
print(f'crown diff : {crown_count}')
print(f'fruit diff : {fruit_count}')
print(f'gnarl diff : {gnarl_count}')
print(f'loc diff : {loc_count}')
print(f'size diff : {size_count}')

branch diff : 12
root diff : 6
crown diff : 7
fruit diff : 17
gnarl diff : 3
loc diff : 12
size diff : 15


In [12]:
branch_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['branch_yn'].iloc[i] != tree_pred_df['branch_yn'].iloc[i]:
        branch_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['branch_yn'].iloc[i], tree_pred_df['branch_yn'].iloc[i]])

print(len(branch_wrong))
for i in branch_wrong:
    print(i)

12
['26_139_23015_tree', 'n', 'y']
['26_140_23028_tree', 'n', 'y']
['26_141_23051_tree', 'n', 'y']
['33_93_23059_tree', 'y', 'n']
['33_96_23126_tree', 'y', 'n']
['33_97_23147_tree', 'n', 'y']
['33_98_23169_tree', 'n', 'y']
['38_104_23005_tree', 'n', 'y']
['3_11_23062_tree', 'n', 'y']
['50_228_23027_tree', 'n', 'y']
['52_215_23038_tree', 'n', 'y']
['5_29_23024_tree', 'n', 'y']


In [13]:
root_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['root_yn'].iloc[i] != tree_pred_df['root_yn'].iloc[i]:
        root_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['root_yn'].iloc[i], tree_pred_df['root_yn'].iloc[i]])

print(len(root_wrong))
for i in root_wrong:
    print(i)

6
['33_94_23086_tree', 'y', 'n']
['33_96_23134_tree', 'n', 'y']
['38_104_23005_tree', 'y', 'n']
['50_227_23012_tree', 'y', 'n']
['50_231_23111_tree', 'n', 'y']
['5_28_23085_tree', 'y', 'n']


In [14]:
crown_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['crown_yn'].iloc[i] != tree_pred_df['crown_yn'].iloc[i]:
        crown_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['crown_yn'].iloc[i], tree_pred_df['crown_yn'].iloc[i]])

print(len(crown_wrong))
for i in crown_wrong:
    print(i)

7
['24_178_23082_tree', 'n', 'y']
['24_181_23158_tree', 'y', 'n']
['33_93_23057_tree', 'y', 'n']
['33_96_23110_tree', 'n', 'y']
['38_104_23010_tree', 'n', 'y']
['52_215_23030_tree', 'n', 'y']
['52_216_23050_tree', 'y', 'n']


In [15]:
fruit_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['fruit_yn'].iloc[i] != tree_pred_df['fruit_yn'].iloc[i]:
        fruit_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['fruit_yn'].iloc[i], tree_pred_df['fruit_yn'].iloc[i]])

print(len(fruit_wrong))
for i in fruit_wrong:
    print(i)

17
['24_178_23082_tree', 'n', 'y']
['24_179_23107_tree', 'n', 'y']
['26_139_23015_tree', 'n', 'y']
['33_93_23057_tree', 'y', 'n']
['33_93_23063_tree', 'n', 'y']
['33_96_23126_tree', 'y', 'n']
['33_96_23134_tree', 'n', 'y']
['33_98_23169_tree', 'n', 'y']
['3_12_23095_tree', 'n', 'y']
['3_13_23111_tree', 'n', 'y']
['47_198_23022_tree', 'n', 'y']
['47_198_23026_tree', 'n', 'y']
['50_227_23012_tree', 'n', 'y']
['52_216_23049_tree', 'y', 'n']
['52_219_23093_tree', 'n', 'y']
['5_28_23085_tree', 'y', 'n']
['5_29_23024_tree', 'n', 'y']


In [16]:
gnarl_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['gnarl_yn'].iloc[i] != tree_pred_df['gnarl_yn'].iloc[i]:
        gnarl_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['gnarl_yn'].iloc[i], tree_pred_df['gnarl_yn'].iloc[i]])

print(len(gnarl_wrong))
for i in gnarl_wrong:
    print(i)

3
['24_178_23070_tree', 'n', 'y']
['33_93_23064_tree', 'y', 'n']
['3_11_23069_tree', 'n', 'y']


In [17]:
loc_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['loc'].iloc[i] != tree_pred_df['loc'].iloc[i]:
        loc_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['loc'].iloc[i], tree_pred_df['loc'].iloc[i]])

print(len(loc_wrong))
for i in loc_wrong:
    print(i)

12
['24_179_23103_tree', 'right', 'center']
['24_179_23107_tree', 'left', 'center']
['24_181_23158_tree', 'left', 'center']
['26_142_23057_tree', 'right', 'center']
['33_96_23126_tree', 'left', 'right']
['33_99_23216_tree', 'left', 'center']
['38_105_23030_tree', 'center', 'left']
['38_106_23060_tree', 'center', 'right']
['3_11_23069_tree', 'left', 'center']
['50_228_23027_tree', 'right', 'center']
['52_218_23115_tree', 'left', 'center']
['52_219_23093_tree', 'left', 'center']


In [18]:
size_wrong = []
for i in range(len(tree_pred_df)): 
    if tree_ori_100_df['size'].iloc[i] != tree_pred_df['size'].iloc[i]:
        size_wrong.append([tree_pred_df['id'].iloc[i], tree_ori_100_df['size'].iloc[i], tree_pred_df['size'].iloc[i]])

print(len(size_wrong))
for i in size_wrong:
    print(i)

15
['24_177_23046_tree', 'big', 'middle']
['24_178_23082_tree', 'middle', 'big']
['24_180_23136_tree', 'small', 'middle']
['26_139_23015_tree', 'small', 'middle']
['33_92_23030_tree', 'middle', 'small']
['33_96_23110_tree', 'small', 'middle']
['33_96_23134_tree', 'big', 'middle']
['33_97_23142_tree', 'small', 'middle']
['33_97_23160_tree', 'big', 'middle']
['33_98_23194_tree', 'small', 'middle']
['3_12_23095_tree', 'middle', 'big']
['47_198_23026_tree', 'small', 'middle']
['50_227_23012_tree', 'middle', 'big']
['52_218_23115_tree', 'small', 'middle']
['5_25_23056_tree', 'big', 'middle']
